In [6]:
!pip install git+https://github.com/BAMresearch/FenicsXConcrete pint gmsh

In [ ]:
import random
from nibelungenbruecke.scripts.digital_twin_orchestrator.interface_test_DT import DigitalTwinInterface
import os

In [ ]:
path = 'use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/settings/digital_twin_default_parameters.json'
# path = '../../../use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/settings/digital_twin_default_parameters.json'
    

In [ ]:
dt = DigitalTwinInterface(path)


In [ ]:
print(dt)

In [ ]:
dt.run_model()